In [536]:
import sys

import numpy as np
import pandas as pd
import mdptoolbox as mtb

import itertools as it
import pprint as pp
from scipy.stats import binom

In [537]:
np.set_printoptions(threshold=sys.maxsize)

In [538]:
# NOTATION

# rplant - renewable plant 
# fplant - fossil fuel plant
# RES - referring to renewable plant
# FF - referring to fossil fuel plant

In [539]:
# VARIABLES

N_YEARS = 10
N_TECHSTAGES = 3
N_PLANTS = 5

# Starting price of carbon per ton.
C_CO2_INIT = 40
# Initial construction costs of a renewable plant per kW (average of solar PV and onshore wind).
C_CAP_RES = [1284, 746, 456]
# Annual operation & maintenance costs of a fossil fuel plant per kW (average of coal and natural gas).
C_OM_FF = 68.8
# Annual emissions of a fossil fuel plant in kg CO2 per kWh (average of coal and natural gas).
FF_EMIT = 2.03

# Probability that tech stage advances to the next given the current stage is not the highest. 
# Assume it is only possible to advance by 1 at a time.
P_ADV_TECHSTAGE = 0.25

# Probability that a renewable plant "fails" at the end of the year.
# A plant that fails is replaced in the next year for the same cost as building a new plant.
RPLANT_LIFE = 25
P_RPLANT_FAIL = 1 / RPLANT_LIFE

# Discount rate (average of solar PV and onshore wind in North America).
DISC_RATE = 0.06

In [541]:
# STATE SPACE

# State space includes:
    # T = Time 
        # Range 2020 to 2050 years
    # V = Tech "stage" 
        # Represents how advanced current energy technologies are
    # N_r = Number of renewable power plants 
        # Out of N total plants
        # Number of fossil fuel plants is N - N_r
        
S = (N_YEARS+1) * (N_TECHSTAGES) * (N_PLANTS+1)

# Create mapping between state and unique integer ID.
def enumerate_states(n_years, n_techstages, n_plants):
    state_to_id = {}
    idx = 0
    iter_states = get_iter_states(n_years, n_techstages, n_plants)
    for state in iter_states:
        (t, v, r) = state
        state_to_id[state] = idx
        idx += 1
    return state_to_id

In [542]:
state_to_id = enumerate_states(N_YEARS, N_TECHSTAGES, N_PLANTS)
id_to_state = {v: k for k, v in state_to_id.items()}
#pp.pprint(state_to_id)

In [543]:
# ACTION SPACE

# Possible actions:
    # 0 -- Do nothing
    # 1...N -- Convert 1...N fossil fuel plants to renewable plants
# An invalid action attempts to convert more fossil fuel plants than remain. 
    
A = N_PLANTS + 1

In [540]:
# COST FUNCTION

def calc_cost(t, v, r, a):
    carbontax = C_CO2_INIT * (1.05 ** t)
    cost_fplants = (N_PLANTS - a) * (C_OM_FF + FF_EMIT * carbontax)
    # Assume renewable plants cost nothing after construction.
    cost_rplants = a * C_CAP_RES[v]
    total = cost_rplants + cost_fplants
    return round(total)

In [615]:
# TRANSITION PROBABILITIES

def trans_probs_wrapper():
    transitions = np.zeros([A, S, S])
    print("Filling transitions probabilities for A = 0 (do nothing)...")
    fill_trans_donothing(transitions, 0)
    print("Filling transitions probabilities for other A...")
    fill_trans_other(transitions)
    print("Transitions done.")
    return transitions

def fill_trans_donothing(transitions, a, state=None):
    iter_states = []
    a_plants = a
    a_action = a
    if state is not None:
        iter_states.append(state)
        a_plants = 0
    else:
        iter_states = get_iter_states(N_YEARS, N_TECHSTAGES, N_PLANTS)
    for state in iter_states:
        (t, v, r), state_curr, idx_curr = breakdown_state(state)
        assert np.sum(transitions[a][idx_curr]) == 0.0, np.sum(transitions[a][idx_curr])
        # Edge case for terminal state.
        if t == N_YEARS:
            transitions[a_action][idx_curr][idx_curr] = 1.0
            continue
        # FAILURE LOOP
        loop_failure(state_curr, transitions, a_plants, a_action)
        assert np.isclose(np.sum(transitions[a_action][idx_curr]), 1.0), np.sum(transitions[a_action][idx_curr])

def fill_trans_other(transitions):
    iter_states = get_iter_states(N_YEARS, N_TECHSTAGES, N_PLANTS)
    for state in iter_states:
        (t, v, r), state_curr, idx_curr = breakdown_state(state)
        # ACTION LOOP
        # From 1 up to number of fossil fuel plants remaining may be converted.
        for a in np.arange(1, A):
            # Transition doesn't matter for final year as long as it exists. 
            if t == N_YEARS:
                transitions[a][idx_curr][idx_curr] = 1.0
                continue
            if a > N_PLANTS-r:
                # Dummy transitions for invalid actions. 
                fill_trans_donothing(transitions, a, state)
            else:
                # FAILURE LOOP
                loop_failure(state_curr, transitions, a, a)
            assert np.isclose(np.sum(transitions[a][idx_curr]), 1.0), np.sum(transitions[a][idx_curr])
            normalize_trans_row(state_curr, transitions, a)

In [631]:
transitions = trans_probs_wrapper()

Filling transitions probabilities for A = 0 (do nothing)...
Filling transitions probabilities for other A...
Transitions done.


In [636]:
# REWARDS MATRIX

def rewards_wrapper():
    rewards = np.zeros([S, A])
    print("Filling rewards...")
    fill_rewards(rewards, S)
    print("Rewards done.")
    return rewards

def fill_rewards(rewards, S):
    for a in np.arange(A):
        for s in np.arange(S):
            state = id_to_state[s]
            idx = state_to_id[state]
            # Sanity check for integer id.
            assert(idx == s)
            (t, v, r) = state
            if a > N_PLANTS-r:
                # For invalid actions, assume cost for highest action possible.
                cost = calc_cost(t, v, r, N_PLANTS-r)
            else:
                cost = calc_cost(t, v, r, a)
            # Model reward as negative cost.
            rewards[idx][a] = -1 * cost
        # Decentivize remaining at state (0, 0, 0).
        rewards[0][0] = -1 * np.inf

In [637]:
rewards = rewards_wrapper()

Filling rewards...
Rewards done.


In [638]:
# HELPER FUNCTIONS

def get_iter_states(n_years, n_techstages, n_plants):
    return it.product(np.arange(n_years+1), np.arange(n_techstages), np.arange(n_plants+1))

def breakdown_state(state):
    (t, v, r) = state
    state_curr = state
    idx_curr = state_to_id[state_curr]
    return (t, v, r), state_curr, idx_curr

def normalize_trans_row(state_curr, transitions, a):
    idx_curr = state_to_id[state_curr]
    transitions[a][idx_curr] = transitions[a][idx_curr] / np.sum(transitions[a][idx_curr])

# Any number of existing renewable plants may fail (at end of year).
def loop_failure(state, transitions, a_plants, a_action): 
    (t, v, r), state_curr, idx_curr = breakdown_state(state)
    for e in np.arange(r+1):
        prob_fail = binom.pmf(e, r, P_RPLANT_FAIL)
        plants_next = r-e 
        if a_plants != 0:
            plants_next += a_plants
        state_next = (t+1, v, plants_next)
        idx_next = state_to_id[state_next]
        if v < N_TECHSTAGES - 1:
            state_next_v = (t+1, v+1, plants_next)
            idx_next_v = state_to_id[state_next_v]
            # Tech stage may remain the same.
            transitions[a_action][idx_curr][idx_next] = (1.0-P_ADV_TECHSTAGE) * prob_fail
            # Tech stage may advance (assume only possible to advance by 1).
            transitions[a_action][idx_curr][idx_next_v] = P_ADV_TECHSTAGE * prob_fail
        else:
            # Tech stage must remain the same.
            transitions[a_action][idx_curr][idx_next] = prob_fail

In [639]:
# UNIT TESTS

def test_trans_prob(state_curr, state_next, action, expected_prob):
    idx_curr = state_to_id[state_curr]
    idx_next = state_to_id[state_next]
    print(state_curr, " to ", state_next, " with action ", action, ":", 
          transitions[action][idx_curr][idx_next], ", ", expected_prob)
    assert(transitions[1][idx_curr][idx_next] == prob_A1)

trans_to_test = []
trans_to_test.append((0, 0, 0), (1, 0, 1), 1, (1.0-P_ADV_TECHSTAGE))

for state_curr, state_next, action, expected_prob in trans_to_test:
    test_trans_prob()

(0, 0, 0)  to  (1, 0, 1)  with action 1:  0.75 ,  0.75


In [650]:
def MDP_wrapper(n_years, n_techstages, n_plants):
    print("Time range: 0 t", n_years)
    print("Number of tech stages: ", n_techstages)
    print("Total plants: ", n_plants, end="\n")
    S = (n_years+1) * (n_techstages) * (n_plants+1)
    A = n_plants+1
    print("Initializing MDP...\n")
    enumerate_states(n_plants, n_techstages, n_years)
    transitions = trans_probs_wrapper()
    rewards = rewards_wrapper()
    mdp_FH = mtb.mdp.FiniteHorizon(transitions, rewards, DISC_RATE, N_YEARS)
    print("\nRunning MDP...")
    mdp_FH.run()
    print("MDP done.\n")
    print("Optimal policy:\n", mdp_FH.policy)
    print("Optimal value function:\n", mdp_FH.V)
    return mdp_FH

In [651]:
MDP_wrapper(N_YEARS, N_TECHSTAGES, N_PLANTS)

Time range: 0 t 10
Number of tech stages:  3
Total plants:  5
Initializing MDP...

Filling transitions probabilities for A = 0 (do nothing)...
Filling transitions probabilities for other A...
Transitions done.
Filling rewards...
Rewards done.

Running MDP...
MDP done.

Optimal value function:
 [[-1933.23858597 -1933.23858596 -1933.2385858  -1933.23858324
  -1933.23854181 -1933.23787152 -1933.227024   -1933.0512
  -1930.2        -1884.             0.        ]
 [ -799.23858597  -799.23858596  -799.2385858   -799.23858324
   -799.23854181  -799.23787152  -799.227024    -799.0512
   -796.2         -750.             0.        ]
 [ -799.23858597  -799.23858596  -799.2385858   -799.23858324
   -799.23854181  -799.23787152  -799.227024    -799.0512
   -796.2         -750.             0.        ]
 [ -799.23858597  -799.23858596  -799.2385858   -799.23858324
   -799.23854181  -799.23787152  -799.227024    -799.0512
   -796.2         -750.             0.        ]
 [ -799.23858597  -799.23858596  

In [ ]:
iter_params = it.product(np.arange(10,51), [3], np.arange(5, 11))
MDP_results = {}

blockPrint()

for t, v, r in iter_params:
    mdp_FH = MDP_wrapper(t, v, r)
    MDP_results[(t, v, r)] = mdp_FH.policy

enablePrint()